# How to use the prebuilt ReAct agent

In this how-to we'll create a simple [ReAct](https://arxiv.org/abs/2210.03629) agent app that can check the weather. The app consists of an agent (LLM) and tools. As we interact with the app, we will first call the agent (LLM) to decide if we should use tools. Then we will run a loop:  

1. If the agent said to take an action (i.e. call tool), we'll run the tools and pass the results back to the agent
2. If the agent did not ask to run tools, we will finish (respond to the user)

<div class="admonition warning">
    <p class="admonition-title">Prebuilt Agent</p>
    <p>
Please note that here will we use a prebuilt agent. One of the big benefits of LangGraph is that you can easily create your own agent architectures. So while it's fine to start here to build an agent quickly, we would strongly recommend learning how to build your own agent so that you can take full advantage of LangGraph.
    </p>
</div>   

## Code

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import random

from botocore.config import Config
config = Config(
    region_name=random.choice(['us-west-2','us-east-1']),
    read_timeout=600, ## Timeout 시간 조정
    retries = dict(
        max_attempts = 8 ## Retry 횟수 조정
    )
)

from langchain_aws import ChatBedrock
## https://api.python.langchain.com/en/latest/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html

model_kwargs = { #anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024, 
    "temperature": 0
}
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", #파운데이션 모델 지정
    model_kwargs=model_kwargs,
    # streaming=True,
    config=config
) #Claude 속성 구성

In [3]:
# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)
from langchain import hub
from typing import Literal
from langchain.memory import ChatMessageHistory
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, Tool, create_react_agent

@tool
# def get_weather(city: Literal["nyc", "sf"]):
def get_weather(city):
    """Use this to get weather information."""
    city = city.rstrip("\n")
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

prompt = hub.pull("hwchase17/react")
memory = ChatMessageHistory(session_id="chat-history")
    
# Define the graph

agent = create_react_agent(model, tools, prompt)

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
                agent_executor,
                lambda session_id: memory,
                input_messages_key="input",
                history_messages_key="chat_history",
            )

inputs = "what is the weather in sf"
agent_with_chat_history.invoke(
    {"input": inputs},
    config={"configurable": {"session_id": "JHJMNBNMB67686"}}
)



> Entering new AgentExecutor chain...
Thought: To find out the weather in San Francisco (sf), I will need to use the provided get_weather tool.

Action: get_weather
Action Input: sf
It's always sunny in sf

In [5]:
memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='what is the weather in sf'), AIMessage(content='According to the get_weather tool, the weather in San Francisco (sf) is always sunny.')])

In [6]:
from langchain_core.runnables.history import RunnableWithMessageHistory

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
                agent_executor,
                lambda session_id: memory,
                input_messages_key="input",
                history_messages_key="chat_history",
            )

inputs = "서울의 날씨는 어때?"
agent_with_chat_history.invoke(
    {"input": inputs},
    config={"configurable": {"session_id": "JHJMNBNMB67686"}}
)



> Entering new AgentExecutor chain...
Thought: 이 질문에 답하기 위해서는 서울의 날씨 정보가 필요합니다.
Action: get_weather(city="서울")
Action Input: 서울get_weather(city="서울") is not a valid tool, try one of [get_weather].Question: 서울의 날씨는 어때?
Thought: 이 질문에 답하기 위해서는 서울의 날씨 정보가 필요합니다.
Action: get_weather
Action Input: 서울

AssertionError: Unknown city